In [ ]:
#pip install pyodbc

In [1]:
import pyodbc
import pandas as pd

In [ ]:
# Database connection parameters
server = 'PT-L163255\SQLEXPRESS01'  # e.g., 'localhost' or 'server_name'
database = 'University'  # e.g., 'my_database'

# Connection string
conn_str = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f'Trusted_Connection=yes;'
)

# Connect to the SQL Server database
conn = pyodbc.connect(conn_str)

# Execute a query and load data into a DataFrame
#query = "SELECT * FROM aluno"  # Replace 'your_table' with your actual table name
#df = pd.read_sql_query(query, conn)

query = f"SELECT * FROM aluno"
cursor = conn.cursor()
cursor.execute(query)
results = [list(row) for row in cursor.fetchall()]
colunas = ['nr_aluno', 'nome', 'apelido', 'curso', 'media_entrada', 'curso_concluido', 'media_final']
df = pd.DataFrame(results, columns= colunas )

In [2]:
df = pd.read_csv('/workspaces/Data_Engineering/output_dataengineering.csv')
df

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
0,0,20031358,Leonardo,Pires,Psicologia,15,True,16.0
1,1,2022158,Raquel,Laranjeira,Informática,17,True,11.0
2,2,2003812,Anne,Silva,Informática,16,True,16.0
3,3,2022901,Patrícia,Castelo-Branco,Informática,11,True,17.0
4,4,2019303,Avelino,Sampaio,Comunicação,13,True,18.0
...,...,...,...,...,...,...,...,...
1755,1755,2019869,Alice,Laranjeira,Informática,11,True,17.0
1756,1756,20221093,Patrícia,Pereira,Comunicação,14,True,16.0
1757,1757,20071160,Lucas,Capicua,Informática,12,True,20.0
1758,1758,2020288,Pedro,Capicua,Comunicação,13,True,19.0


### Check Duplicados

In [3]:
#Não há duplicados
duplicados = df['nr_aluno'].duplicated()
duplicados.sum()

np.int64(0)

### Collect all 'Apelido' that contains '()'

In [4]:
#Apelido retirar os parenteses
#df['apelido'] = df['apelido'].str.replace(r'[()]', '', regex=True)
df_apelido = df[df['apelido'].str.contains(r'\(.*\)')]
df_apelido

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
100,100,2210000,Thomas Anderson,(Neo),Informática,20,True,20.0
201,201,2290110,Anakin Skywalker,(Lord Vader),Psicologia,20,True,20.0
804,804,2210030,Sininho,(Terra do Nunca),Cintilar,-20,True,-20.0


### Select that are not in the list [['Psicologia', 'Informática', 'Comunicação']]

In [56]:
cursos = ['Psicologia', 'Informática', 'Comunicação']
df.loc[~df['curso'].isin(cursos)] = 'NULL'

#df.curso.isnull()
filtered_df_curso_null = df[df['curso'] == 'NULL']
filtered_df_curso_null

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
26,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
28,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
31,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
35,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
36,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
37,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
41,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
43,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
44,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
51,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [50]:
df['media_entrada'] = pd.to_numeric(df['media_entrada'], errors='coerce')
filtered_df_entrada= df[(df['media_entrada'] < 0) | (df['media_entrada'].isnull())]
filtered_df_entrada

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
26,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
28,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
31,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
35,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
36,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
37,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
41,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
43,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
44,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
51,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN


In [52]:
df['media_final'] = pd.to_numeric(df['media_final'], errors='coerce')
filtered_df_final = df[(df['media_final'] < 0) | (df['media_final'].isnull())]
filtered_df_final

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
25,25,2003821,Nuno,Laranjeira,Psicologia,16.0,False,NaN
26,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
27,27,200739,Alice,Pires,Psicologia,17.0,False,NaN
28,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
30,30,2003485,Alice,Pires,Psicologia,13.0,False,NaN
31,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
33,33,20211052,Raquel,Castelo,Comunicação,16.0,False,NaN
35,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
36,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
37,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN


In [87]:
error_dataframe = pd.concat([df_apelido, filtered_df_curso_null, filtered_df_entrada ,filtered_df_final], ignore_index=True)
df_no_duplicates =  error_dataframe.drop_duplicates()
df_no_duplicates_reset = df_no_duplicates.reset_index(drop=True)

df_no_duplicates_reset

,0,nr_aluno,nome,apelido,curso,media_entrada,curso_concluido,media_final
0,100,2210000,Thomas Anderson,(Neo),Informática,20,True,20.0
1,201,2290110,Anakin Skywalker,(Lord Vader),Psicologia,20,True,20.0
2,804,2210030,Sininho,(Terra do Nunca),Cintilar,-20,True,-20.0
3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,NULL,NULL,NULL,NULL,NULL,NaN,NULL,NaN
5,79,20201046,Miguel,Silva,Psicologia,-1.0,True,17.0
6,85,2019514,Diana,Sampaio,Informática,-2.0,True,12.0
7,25,2003821,Nuno,Laranjeira,Psicologia,16.0,False,NaN
8,27,200739,Alice,Pires,Psicologia,17.0,False,NaN
9,30,2003485,Alice,Pires,Psicologia,13.0,False,NaN


In [88]:
df_no_duplicates.duplicated().sum()

np.int64(0)

In [90]:
# Convert DataFrame to a text file
error_dataframe.to_csv('fase_1.txt', sep='\t', index=False)


In [ ]:

# Write DataFrame to a CSV file
#df.to_csv('output_db.csv', index=False)

# Close the database connection
conn.close()
